## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [20]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [22]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), 
                                 activation='relu',
                                 input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

# 池化
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/10
50000/50000 [==============================] - 83s 2ms/step - loss: 1.3944 - acc: 0.5368
Epoch 2/10
50000/50000 [==============================] - 82s 2ms/step - loss: 0.8966 - acc: 0.6888
Epoch 3/10
50000/50000 [==============================] - 82s 2ms/step - loss: 0.7152 - acc: 0.7509
Epoch 4/10
50000/50000 [==============================] - 81s 2ms/step - loss: 0.5832 - acc: 0.7961
Epoch 5/10
50000/50000 [==============================] - 81s 2ms/step - loss: 0.4713 - acc: 0.8335
Epoch 6/10
50000/50000 [==============================] - 81s 2ms/step - loss: 0.3668 - acc: 0.8719
Epoch 7/10
50000/50000 [==============================] - 81s 2ms/step - loss: 0.2849 - acc: 0.9010
Epoch 8/10
50000/50000 [==============================] - 81s 2ms/step - loss: 0.2361 - acc: 0.9177
Epoch 9/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.1833 - acc: 0.9368
Epoch 10/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.1662 - acc: 0.94

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [23]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.4897882e-04, 2.2113303e-08, 4.2117783e-01, 5.6185299e-01,
        1.4116615e-02, 6.3313346e-05, 1.9381330e-03, 2.0679095e-04,
        3.9136613e-04, 4.0352575e-06]], dtype=float32)